# TECH CHALLENGE FIAP - 2024

Imagine agora, que você vai atuar como Expert em Data Analytics em uma empresa que exporta vinhos do Brasil para o mundo todo.

Sua área é recém-criada dentro da empresa, e você será responsável pelos relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

1. Dados climáticos
2. Dados demográficos
3. Dados econômicos
4. Dados de avaliação de vinhos

O Head de Dados pediu para que você construísse uma tabela contendo as seguintes informações:

1. País de origem
2. País de destino
3. Quantidade (L)
4. Valor em US$

Os dados que lhe forneceram são de uma vinícola parceira, e podem ser encontrados <a href="http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01" target="_blank">aqui</a>.

Agora, você precisa construir um relatório com as informações solicitadas.

Seu objetivo é dizer o montante de venda de exportação nos últimos 15 anos, separando a análise por país e trazendo quais as prospecções futuras e possíveis ações para uma melhoria nas exportações. Construa gráficos atraentes e que passem a ideia central para que os acionistas e investidores possam seguir em frente com suas ações.

## Análise Explotatória de Dados

In [2]:
import pandas as pd
import openpyxl

In [3]:
export_data = pd.read_csv("../data/ExpVinho.csv", sep=";")

In [4]:
export_data.shape

(128, 108)

In [5]:
export_data.head(-5)

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,26,95,4,21,0,0,0,0
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
3,4,Angola,0,0,0,0,0,0,0,0,...,477,709,345,1065,0,0,0,0,4068,4761
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,119,"Tcheca, República",0,0,0,0,0,0,0,0,...,2712,20980,2115,16391,563,4805,456,5988,1305,9997
119,120,Togo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1890,2012,17317,25608
120,121,Trinidade Tobago,0,0,0,0,0,0,336,300,...,0,0,0,0,0,0,0,0,0,0
121,122,Tunísia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Transformação dos dados

In [6]:
export_data.columns[-32:]

Index(['2007', '2007.1', '2008', '2008.1', '2009', '2009.1', '2010', '2010.1',
       '2011', '2011.1', '2012', '2012.1', '2013', '2013.1', '2014', '2014.1',
       '2015', '2015.1', '2016', '2016.1', '2017', '2017.1', '2018', '2018.1',
       '2019', '2019.1', '2020', '2020.1', '2021', '2021.1', '2022', '2022.1'],
      dtype='object')

In [7]:
# Selecionando as colunas de interesse para os dados de exportação de vinhos

total_columns = len(export_data.columns)
selected_export_data = export_data.iloc[
    :, [0, 1] + list(range(total_columns - 32, total_columns))
]
selected_export_data.head()

,Id,País,2007,2007.1,2008,2008.1,2009,2009.1,2010,2010.1,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,26,95,4,21,0,0,0,0
2,3,"Alemanha, República Democrática",119512,238052,265742,429970,225086,393482,27715,138666,...,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
3,4,Angola,25931,49753,25721,71083,54786,84235,33557,189891,...,477,709,345,1065,0,0,0,0,4068,4761
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


🚨 Podemos notar que a coluna terminada em .1 se refere ao valor monetário (US$) e a coluna que apresenta somente o ano se refere a quantidade em (Kg). 

Deste modo podemos separar nossos dados em duas tabelas diferentes, uma para quantidade e outra para valor.

In [8]:
# Filtrar as colunas para quantidade exportada
exported_quantity = selected_export_data.melt(
    id_vars=["Id", "País"], var_name="Ano", value_name="exported_quantity"
)
exported_quantity = exported_quantity[
    exported_quantity["Ano"].apply(lambda x: not x.endswith(".1"))
]

# Filtrar as colunas para valor exportado
exported_value = selected_export_data.melt(
    id_vars=["Id", "País"], var_name="Ano", value_name="exported_value"
)
exported_value = exported_value[
    exported_value["Ano"].apply(lambda x: x.endswith(".1"))
]

# Remover o sufixo '.1' dos anos em valor exportado
exported_value["Ano"] = exported_value["Ano"].str.replace(".1", "")

In [9]:
exported_value.head()

,Id,País,Ano,exported_value
128,1,Afeganistão,2007,0
129,2,África do Sul,2007,0
130,3,"Alemanha, República Democrática",2007,238052
131,4,Angola,2007,49753
132,5,Anguilla,2007,0


In [10]:
exported_quantity.head()

,Id,País,Ano,exported_quantity
0,1,Afeganistão,2007,0
1,2,África do Sul,2007,0
2,3,"Alemanha, República Democrática",2007,119512
3,4,Angola,2007,25931
4,5,Anguilla,2007,0


In [11]:
merged_data = pd.merge(exported_quantity, exported_value, on=["Id", "País", "Ano"])
merged_data.rename(
    columns={"País": "País de destino", "exported_quantity": "Quantidade (kg/L)", "exported_value": "Valor em US$"},
    inplace=True,
)
merged_data["País de origem"] = "Brasil"
order = ["Id", "País de origem", "País de destino", "Ano", "Quantidade (kg/L)", "Valor em US$"]
merged_data = merged_data[order]
merged_data.head()

,Id,País de origem,País de destino,Ano,Quantidade (kg/L),Valor em US$
0,1,Brasil,Afeganistão,2007,0,0
1,2,Brasil,África do Sul,2007,0,0
2,3,Brasil,"Alemanha, República Democrática",2007,119512,238052
3,4,Brasil,Angola,2007,25931,49753
4,5,Brasil,Anguilla,2007,0,0


In [12]:
merged_data.to_csv("../data/exported_data2007_2022.csv", index=False)

## Levantando, tratando e incluindo Dados de Tamanho da População acima de 18 anos

In [13]:
# Fazendo a leitura de dados demográficos sobre o tamanho da população por idade de cada país

# population_by_age = pd.read_excel('../data/population_by_age.xlsx', skiprows=16, header=0, index_col='Index')
# valores = population_by_age['Region, subregion, country or area *'].unique()

In [14]:
# Filtrando apenas os países, sem os grupos regionais

# population_by_age = population_by_age.loc[population_by_age['Type']=='Country/Area']

In [15]:
# id_vars = ['Variant', 'Region, subregion, country or area *', 'Notes', 'Location code', 'ISO3 Alpha-code', 'ISO2 Alpha-code', 'SDMX code**', 'Type', 'Parent code', 'Year']
# population_by_age = population_by_age.melt(id_vars=id_vars, var_name='Age', value_name='Population Qt')

In [16]:
# population_by_age = population_by_age[["Region, subregion, country or area *", "Year", "Age", "Population Qt"]]

In [17]:
# population_by_age.rename(columns={'Region, subregion, country or area *': 'País', 'Year': 'Ano', 'Age': 'Idade'}, inplace=True)

In [18]:
# Filtrando apenas a população acima de 18 anos que é a idade legal permitidada para beber álcool na maioria dos países 

# population_by_age.query('Idade == 18')

,País,Ano,Idade,Population Qt
305856,Burundi,1950.0,18,44.357
305857,Burundi,1951.0,18,44.923
305858,Burundi,1952.0,18,45.4555
305859,Burundi,1953.0,18,45.999
305860,Burundi,1954.0,18,46.727
...,...,...,...,...
322843,Wallis and Futuna Islands,2017.0,18,0.1745
322844,Wallis and Futuna Islands,2018.0,18,0.1865
322845,Wallis and Futuna Islands,2019.0,18,0.1905
322846,Wallis and Futuna Islands,2020.0,18,0.1875


In [19]:
# population_by_age['Idade'].apply(type).unique()

array([<class 'int'>, <class 'str'>], dtype=object)

In [20]:
# population_by_age['Ano'].apply(type).unique()

array([<class 'float'>], dtype=object)

In [21]:
# population_by_age['Idade'] = population_by_age['Idade'].replace('100+', 100)
# population_by_age['Idade'] = [int(x) if isinstance(x, int) or (isinstance(x, str) and x.isdigit()) else 0 for x in population_by_age['Idade']]

/var/folders/s8/p68_rlhj54bb1q_xvzdsc1m40000gn/T/ipykernel_19997/3500055615.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  population_by_age['Idade'] = population_by_age['Idade'].replace('100+', 100)


In [22]:
# population_by_age['Ano'] = population_by_age['Ano'].astype(int)

In [23]:
# population_by_age = population_by_age.query('Idade >= 18 and Ano >= 2007')

In [24]:
# population_by_age_countries = set(population_by_age['País'].unique())
# merged_data_countries = set(merged_data['País de destino'].unique())

# diference_merged_data = merged_data_countries - population_by_age_countries
# diference_population_by_age = population_by_age_countries - merged_data_countries

In [25]:
# print('Países em tabela de exportação, mas não em tabela de população: ', sorted(diference_merged_data))

Países em tabela de exportação, mas não em tabela de população:  ['Afeganistão', 'Alemanha, República Democrática', 'Antilhas Holandesas', 'Antígua e Barbuda', 'Austrália', 'Barein', 'Belice', 'Bolívia', 'Brasil', 'Bulgária', 'Bélgica', 'Bósnia-Herzegovina', 'Camarões', 'Canadá', 'Catar', 'Cayman, Ilhas', 'Chipre', 'Cingapura', 'Cocos (Keeling), Ilhas', 'Colômbia', 'Comores', 'Coreia, Republica Sul', 'Costa do Marfim', 'Croácia', 'Dinamarca', 'Emirados Arabes Unidos', 'Equador', 'Eslovaca, Republica', 'Espanha', 'Estados Unidos', 'Estônia', 'Filipinas', 'Finlândia', 'França', 'Gana', 'Granada', 'Grécia', 'Guiana', 'Guiana Francesa', 'Guine Bissau', 'Guine Equatorial', 'Hong Kong', 'Hungria', 'Ilha de Man', 'Ilhas Virgens', 'Indonésia', 'Iraque', 'Irlanda', 'Irã', 'Itália', 'Japão', 'Jordânia', 'Letônia', 'Libéria', 'Luxemburgo', 'Líbano', 'Malásia', 'Marshall, Ilhas', 'Mauritânia', 'Moçambique', 'México', 'Nigéria', 'Noruega', 'Nova Caledônia', 'Nova Zelândia', 'Omã', 'Panamá', 'Paragu

In [26]:
# print('Países em tabela de população, mas não em tabela de exportação: ', sorted(diference_population_by_age))

Países em tabela de população, mas não em tabela de exportação:  ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Antigua and Barbuda', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Belarus', 'Belgium', 'Belize', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, Taiwan Province of China', 'Colombia', 'Comoros', 'Cook Islands', 'Croatia', 'Cyprus', 'Czechia', "Côte d'Ivoire", "Dem. People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', '

In [27]:
mapeamento_correcoes = {
    'Afghanistan': 'Afeganistão',
    'South Africa': 'África do Sul',
    'German Democratic Republic': 'Alemanha, República Democrática',
    'Australia': 'Austrália',
    'Austria': 'Áustria',
    'Bahrain': 'Barein',
    'Belize': 'Belice',
    'Bolivia (Plurinational State of)': 'Bolívia',
    'Brazil': 'Brasil',
    'Bulgaria': 'Bulgária',
    'Belgium': 'Bélgica',
    'Bosnia and Herzegovina': 'Bósnia-Herzegovina',
    'Cameroon': 'Camarões',
    'Canada': 'Canadá',
    'Qatar': 'Catar',
    'Cayman Islands': 'Cayman, Ilhas',
    'Cyprus': 'Chipre',
    'Singapore': 'Cingapura',
    'Hong Kong': 'Hong Kong',
    'Colombia': 'Colômbia',
    'Comoros': 'Comores',
    'Republic of Korea': 'Coreia, Republica Sul',
    "Côte d'Ivoire": 'Costa do Marfim',
    'Croatia': 'Croácia',
    'Denmark': 'Dinamarca',
    'United Arab Emirates': 'Emirados Arabes Unidos',
    'Ecuador': 'Equador',
    'Slovakia': 'Eslovaca, Republica',
    'Spain': 'Espanha',
    'United States of America': 'Estados Unidos',
    'Estonia': 'Estônia',
    'Philippines': 'Filipinas',
    'Finland': 'Finlândia',
    'France': 'França',
    'Ghana': 'Gana',
    'Greece': 'Grécia',
    'Guinea-Bissau': 'Guine Bissau',
    'Equatorial Guinea': 'Guine Equatorial',
    'Hungary': 'Hungria',
    'Iraq': 'Iraque',
    'Ireland': 'Irlanda',
    'Iran (Islamic Republic of)': 'Irã',
    'Italy': 'Itália',
    'Japan': 'Japão',
    'Jordan': 'Jordânia',
    'Latvia': 'Letônia',
    'Liberia': 'Libéria',
    'Lebanon': 'Líbano',
    'Malaysia': 'Malásia',
    'Marshall Islands': 'Marshall, Ilhas',
    'Mauritania': 'Mauritânia',
    'Mexico': 'México',
    'Nigeria': 'Nigéria',
    'Norway': 'Noruega',
    'New Caledonia': 'Nova Caledônia',
    'New Zealand': 'Nova Zelândia',
    'Oman': 'Omã',
    'Panama': 'Panamá',
    'Paraguay': 'Paraguai',
    'Netherlands': 'Países Baixos',
    'Poland': 'Polônia',
    'Puerto Rico': 'Porto Rico',
    'Kenya': 'Quênia',
    'United Kingdom': 'Reino Unido',
    'Dominican Republic': 'República Dominicana',
    'Russian Federation': 'Rússia',
    'Sierra Leone': 'Serra Leoa',
    'Sweden': 'Suécia',
    'Switzerland': 'Suíça',
    'Sao Tome and Principe': 'São Tomé e Príncipe',
    'Saint Vincent and the Grenadines': 'São Vicente e Granadinas',
    'Thailand': 'Tailândia',
    'Taiwan Province of China': 'Taiwan (FORMOSA)',
    'United Republic of Tanzania': 'Tanzânia',
    'Czechia': 'Tcheca, República',
    'Trinidad and Tobago': 'Trinidade Tobago',
    'Tunisia': 'Tunísia',
    'Turkey': 'Turquia',
    'Uruguay': 'Uruguai',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Vietnam': 'Vietnã',
}

In [28]:
# population_by_age['País'] = population_by_age['País'].replace(mapeamento_correcoes)

In [29]:
# merged_data['Ano'] = merged_data['Ano'].astype(int)

In [30]:
# population_by_age = population_by_age.groupby(by=['País', 'Ano'], as_index=False).sum()

In [31]:
# population_by_age['Population Qt'].apply(type).unique()

array([<class 'float'>], dtype=object)

In [32]:
# population_by_age.drop('Idade', axis=1, inplace=True)

In [33]:
# population_by_age.rename(columns={'País': 'País de destino'}, inplace=True)
# merged_data2 = pd.merge(merged_data, population_by_age, on=['Ano', 'País de destino'], how='left')

In [34]:
# merged_data2[merged_data2['País de destino'] == 'Paraguai']


,Id,País de origem,País de destino,Ano,Quantidade (kg/L),Valor em US$,Population Qt
97,98,Brasil,Paraguai,2007,1285459,801519,3266.697
225,98,Brasil,Paraguai,2008,2191901,1374088,3336.826
353,98,Brasil,Paraguai,2009,486927,392087,3408.948
481,98,Brasil,Paraguai,2010,510989,449197,3486.602
609,98,Brasil,Paraguai,2011,240168,276281,3570.124
737,98,Brasil,Paraguai,2012,354824,428279,3656.327
865,98,Brasil,Paraguai,2013,481564,680828,3743.547
993,98,Brasil,Paraguai,2014,521847,908028,3830.464
1121,98,Brasil,Paraguai,2015,495428,741370,3916.6955
1249,98,Brasil,Paraguai,2016,985739,1655417,4002.1165


In [38]:
# merged_data2.reset_index(drop=True, inplace=True)

In [41]:
# merged_data2.to_csv('../data/exported_data_2.csv', index=False)